In [1]:
import numpy as np
from keras.datasets import imdb
from tensorflow.python.keras.models import Model, load_model, Sequential
from tensorflow.python.keras.layers import LSTM, Dense, Embedding, Input, Dot, Flatten, Layer, Lambda
from keras_preprocessing.sequence import pad_sequences
from tensorflow.python.keras.callbacks import ModelCheckpoint
import tensorflow as tf
from keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
import pandas as pd
import pickle
import re
import os
from keras.utils import to_categorical
from datetime import datetime

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
placement_weights = np.array([1, 0.875, 0.75, 0.625, 0.375, 0.25, 0.125, 0])

In [4]:
with open('embeddings_100.pkl', 'rb') as file:
    weights = pickle.load(file)
word2idx = {}
with open('word2idx.pkl', 'rb') as file:
    word2idx = pickle.load(file)

In [5]:
sentences = []
with open('teams.txt', 'r') as file:
    for line in file:
        sentences.append(line.strip().split())

In [7]:
maxWords = 0
for i in range(len(sentences)):
    if maxWords < len(sentences[i]):
        maxWords = len(sentences[i])

In [8]:
X = np.zeros((len(sentences), maxWords - 1, 100))
Y = np.zeros((len(sentences)))
for i in range(len(sentences)):
    for j in range(len(sentences[i])):
        if sentences[i][j] == 'nan':
            X[i][j] = np.zeros((100,))
        else:
            if j == len(sentences[i]) - 1:
                Y[i] = placement_weights[int(sentences[i][j]) - 1]
            else:
                if str(sentences[i][j]).isdigit():
                    X[i][j] = np.ones((100,)) * int(sentences[i][j])
                else:
                    X[i][j] = weights[word2idx[sentences[i][j]] - 1]

In [9]:
X_train, X_rest, Y_train, Y_rest = train_test_split(X, Y, test_size=0.1, random_state=42)
X_test, X_val, Y_test, Y_val = train_test_split(X_rest, Y_rest, test_size=0.5, random_state=42)

In [11]:
rnn_model = Sequential()
rnn_model.add(LSTM(200, dropout=0.5, recurrent_dropout=0.5))
rnn_model.add(Dense(1024, activation='relu'))
rnn_model.add(Dense(512, activation='relu'))
rnn_model.add(Dense(256, activation='relu'))
rnn_model.add(Dense(128, activation='relu'))
rnn_model.add(Dense(64, activation='relu'))
rnn_model.add(Dense(32, activation='relu'))
rnn_model.add(Dense(16, activation='relu'))
rnn_model.add(Dense(1))

rnn_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse', 'accuracy'])

checkpoint = []
date = ('_' + str(datetime.now().day) + '-' + str(datetime.now().month) + '-' + str(datetime.now().year) + '_' +
        str(datetime.now().hour) + '-' + str(datetime.now().minute).zfill(2))
checkpoint.append(ModelCheckpoint(filepath='rnn_tft_val_mse_' + date + '.h5',
                             monitor='val_mse',
                             verbose=1,
                             save_best_only=True
                             ))
checkpoint.append(ModelCheckpoint(filepath='rnn_tft_val_loss_' + date + '.h5',
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True
                             ))

In [13]:
rnn_model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=30, batch_size=4096, callbacks=checkpoint)

Epoch 1/30
15/15 [==============================] - 89s 6s/step - loss: 0.2056 - mse: 0.2056 - accuracy: 0.1352 - val_loss: 0.1260 - val_mse: 0.1260 - val_accuracy: 0.1191

Epoch 00001: val_mse improved from inf to 0.12600, saving model to rnn_tft_val_mse__8-5-2024_4-05.h5

Epoch 00001: val_loss improved from inf to 0.12600, saving model to rnn_tft_val_loss__8-5-2024_4-05.h5
Epoch 2/30
15/15 [==============================] - 90s 6s/step - loss: 0.1204 - mse: 0.1204 - accuracy: 0.1262 - val_loss: 0.1177 - val_mse: 0.1177 - val_accuracy: 0.1257

Epoch 00002: val_mse improved from 0.12600 to 0.11765, saving model to rnn_tft_val_mse__8-5-2024_4-05.h5

Epoch 00002: val_loss improved from 0.12600 to 0.11765, saving model to rnn_tft_val_loss__8-5-2024_4-05.h5
Epoch 3/30
15/15 [==============================] - 96s 6s/step - loss: 0.1111 - mse: 0.1111 - accuracy: 0.1551 - val_loss: 0.1018 - val_mse: 0.1018 - val_accuracy: 0.1958

Epoch 00003: val_mse improved from 0.11765 to 0.10180, saving m

In [11]:
rnn_model.evaluate(X_test, Y_test, verbose=1)

106/106 [==============================] - 3s 31ms/step - loss: 0.0706 - mse: 0.0706 - accuracy: 0.2342


[0.07061946392059326, 0.07061946392059326, 0.23417533934116364]

In [51]:
worse_team = 'TFT11_Malphite 2 nan nan nan TFT11_KhaZix 2 nan nan nan TFT11_Kindred 2 TFT11_Item_HeavenlyEmblemItem nan nan TFT11_Neeko 2 TFT_Item_SpectralGauntlet TFT_Item_Crownguard nan TFT11_Qiyana 2 nan nan nan TFT11_Soraka 2 TFT_Item_Morellonomicon TFT_Item_GuinsoosRageblade nan TFT11_Yone 3 TFT_Item_UnstableConcoction TFT_Item_Bloodthirster TFT_Item_TitansResolve TFT11_Tristana 1 nan nan nan TFT11_Kayn 2 TFT_Item_Deathblade TFT_Item_Bloodthirster TFT_Item_UnstableConcoction TFT11_Augment_HeavenlyCrest TFT9_Augment_YouHaveMySword TFT9_Augment_BigGrabBag'

In [52]:
worse_team = worse_team.split()
vector = np.zeros((1,len(worse_team), 100))
for i in range(len(worse_team)):
    if worse_team[i] == 'nan':
        vector[0][i] = np.zeros((100,))
    elif worse_team[i].isdigit():
        vector[0][i] = np.ones((100,)) * int(worse_team[i])
    else:
        vector[0][i] = weights[word2idx[worse_team[i]] - 1]

In [53]:
rnn_model.predict(vector)

array([[0.35351145]], dtype=float32)

In [24]:
team1 = 'TFT11_Caitlyn nan nan nan TFT11_Lux TFT_Item_RapidFireCannon TFT_Item_HextechGunblade nan TFT11_Shen nan nan nan TFT11_Ashe TFT_Item_GuinsoosRageblade TFT_Item_HextechGunblade TFT_Item_MadredsBloodrazor TFT11_Ornn TFT_Item_ThiefsGloves TFT_Item_PowerGauntlet TFT_Item_LastWhisper TFT11_Nautilus nan nan nan TFT11_Sett TFT_Item_AdaptiveHelm TFT_Item_Redemption nan TFT11_Lissandra TFT_Item_Leviathan TFT_Item_JeweledGauntlet TFT_Item_RecurveBow TFT11_Hwei nan nan nan TFT11_Augment_WanderingTrainerGold TFT9_Augment_Commander_Ascension TFT10_Augment_InspiringEpitaph'


In [25]:
team1 = team1.split()
vector = np.zeros((1,len(team1), 100))
for i in range(len(team1)):
    if team1[i] == 'nan':
        vector[0][i] = np.zeros((100,))
    else:
        vector[0][i] = weights[word2idx[team1[i]] - 1]

In [26]:
rnn_model.predict(vector)

array([[1.0332756]], dtype=float32)

In [18]:
team5 = 'TFT11_Caitlyn nan nan nan TFT11_Aatrox nan nan nan TFT11_Shen nan nan nan TFT11_Illaoi TFT_Item_Redemption TFT_Item_GargoyleStoneplate TFT_Item_WarmogsArmor TFT11_Morgana TFT_Item_SpearOfShojin TFT5_Item_ArchangelsStaffRadiant TFT_Item_HextechGunblade TFT11_Kayn nan nan nan TFT11_Sylas nan nan nan TFT11_Sett TFT_Item_GuinsoosRageblade TFT_Item_GuardianAngel nan TFT11_Irelia TFT_Item_RapidFireCannon TFT_Item_LastWhisper TFT_Item_InfinityEdge TFT7_Augment_BirthdayPresents TFT9_Augment_Harmacist2 TFT9_Augment_YoungAndWildAndFree'

In [19]:
team5 = team5.split()
vector = np.zeros((1,len(team5), 100))
for i in range(len(team5)):
    if team5[i] == 'nan':
        vector[0][i] = np.zeros((100,))
    else:
        vector[0][i] = weights[word2idx[team5[i]] - 1]

In [20]:
rnn_model.predict(vector)


array([[0.971196]], dtype=float32)

In [6]:
sentences[0]

['TFT11_Caitlyn',
 '2',
 'nan',
 'nan',
 'nan',
 'TFT11_Jax',
 '1',
 'nan',
 'nan',
 'nan',
 'TFT11_Senna',
 '3',
 'TFT5_Item_GuardianAngelRadiant',
 'TFT_Item_HextechGunblade',
 'TFT_Item_RapidFireCannon',
 'TFT11_Shen',
 '2',
 'nan',
 'nan',
 'nan',
 'TFT11_Illaoi',
 '2',
 'TFT_Item_IonicSpark',
 'TFT_Item_FrozenHeart',
 'TFT_Item_Crownguard',
 'TFT11_Yone',
 '1',
 'nan',
 'nan',
 'nan',
 'TFT11_Morgana',
 '1',
 'TFT_Item_AdaptiveHelm',
 'nan',
 'nan',
 'TFT11_Kayn',
 '2',
 'TFT_Item_UnstableConcoction',
 'TFT_Item_GuardianAngel',
 'nan',
 'TFT6_Augment_TinyTitans',
 'TFT10_Augment_CrashTestDummies',
 'TFT9_Augment_HighEndSector',
 '8']